In [1]:
%load_ext autoreload
%autoreload 2
import batcho

In [2]:
def cmd1_prepare(job):
    print('preparing job')
    
def cmd1_run(job):
    print('testing')
    print(job)
    return dict(the_job=job)

batcho.register_job_command(
    command='cmd1',
    prepare=cmd1_prepare,
    run=cmd1_run
)

In [3]:
batcho.set_batch(batch_name='batch1',jobs=[dict(command='cmd1',label='job1',name='first_job'),dict(command='cmd1',label='job2',name='second_job')])

In [4]:
batcho.prepare_batch(batch_name='batch1',clear_jobs=True)
batcho.run_batch(batch_name='batch1')
batcho.assemble_batch(batch_name='batch1')

Retrieving batch batch1
Batch has 2 jobs.
Preparing job job1
preparing job
Preparing job job2
preparing job
Prepared 2 jobs.
Retrieving batch batch1
Batch has 2 jobs.
Acquired lock for job job1
Running job job1
Acquired lock for job job2
Running job job2
Ran 2 jobs.
Retrieving batch batch1
Batch has 2 jobs.
ASSEMBLING job result for job1
ASSEMBLING job result for job2
Assembling 2 results


In [5]:
batcho.get_batch_job_statuses(batch_name='batch1')

[{'job': {'command': 'cmd1', 'label': 'job1', 'name': 'first_job'},
  'status': 'finished'},
 {'job': {'command': 'cmd1', 'label': 'job2', 'name': 'second_job'},
  'status': 'finished'}]

In [11]:
str=batcho.get_batch_job_console_output(batch_name='batch1',job_index=0)
print(str)

testing
{'command': 'cmd1', 'label': 'job1', 'name': 'first_job'}



In [30]:
import time
from kbucket import client as kb
from pairio import client as pa
def _safe_load_object(key):
    try:
        obj=kb.loadObject(key=key)
    except:
        obj=None
    return obj

def listen_for_batch_run_command(key):
    while True:
        obj=_safe_load_object(key=key)
        if obj and ('batch_name' in obj):
            batch_name=obj['batch_name']
            print('Received request to run batch: '+batch_name)
            pa.set(key=key,value=None)
            clear_jobs=obj.get('clear_jobs',True)
            batcho.prepare_batch(batch_name=batch_name,clear_jobs=clear_jobs,stop_key=key)
            batcho.run_batch(batch_name=batch_name,stop_key=key)
            batcho.assemble_batch(batch_name=batch_name,stop_key=key)
        time.sleep(5)

In [31]:
listen_for_batch_run_command(key='listener1')

Received request to run batch: batch1


TypeError: prepare_batch() got an unexpected keyword argument 'stop_key'